In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.091583,"""a""",0.027516,0.025068,0.073583,0.610364,0.579474,-0.098684,0.007555
0.841471,1,"""a""",1,0.585465,"""a""",0.947079,0.917548,0.539384,0.248219,0.909225,-0.391686,0.482099
0.909297,2,"""a""",1,0.098363,"""a""",0.198841,0.113598,0.866751,0.52313,0.392237,-1.236188,-0.009659
0.14112,3,"""a""",1,0.03237,"""a""",0.443498,0.208141,0.137899,0.351743,0.354237,-0.077879,0.137624
-0.756802,4,"""a""",1,0.324095,"""a""",0.804234,0.103371,0.885442,0.413473,0.870453,-1.176478,0.062609


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.610364,-0.579474
-0.841471,-0.248219,-0.909225
-0.909297,-0.52313,-0.392237
-0.14112,-0.351743,-0.354237
0.756802,0.196891,-0.290979


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.498886, -0.35376]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.498886, -0.35376]","[0.086782, 0.406454]"


In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.219035,0.001699,128.887987,0.0,0.215704,0.222366
"""exp(x2)""",0.173641,0.000686,253.137788,0.0,0.172296,0.174986
"""sin(x3)""",-1.743404,0.001351,-1290.252947,0.0,-1.746052,-1.740755
"""__bias__""",-0.106227,0.001517,-70.020282,0.0,-0.109201,-0.103253


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.508478, -0.340153]"
"""a""","[-0.508478, -0.340153]"
"""a""","[-0.508478, -0.340153]"
"""a""","[-0.508478, -0.340153]"
"""a""","[-0.508478, -0.340153]"
…,…
"""b""","[-0.489381, -0.36711]"
"""b""","[-0.489381, -0.36711]"
"""b""","[-0.489381, -0.36711]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.027516,0.025068,-0.098684,-0.022595,-0.076089
0.947079,0.917548,-0.391686,-0.797076,0.405389
0.198841,0.113598,-1.236188,-0.139386,-1.096802
0.443498,0.208141,-0.077879,-0.294887,0.217008
0.804234,0.103371,-1.176478,-0.437789,-0.738689


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.489381, -0.36711]"
"""a""","[-0.508478, -0.340153]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.343272, -0.157735]"
"""b""","[-0.315854, -0.193592]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.54074
"""b""",-0.548295


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.098684,0.027516,0.025068,null,null
-0.391686,0.947079,0.917548,null,null
-1.236188,0.198841,0.113598,null,null
-0.077879,0.443498,0.208141,null,null
-1.176478,0.804234,0.103371,"[-1.609748, 1.186046]",-1.172012
…,…,…,…,…
0.237618,0.71738,0.729978,"[-0.418405, -0.473687]",-0.645937
-0.879749,0.388987,0.291635,"[-0.813367, -0.190164]",-0.371848
-0.302075,0.00809,0.953496,"[-0.891931, -0.00105]",-0.008217


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[28.850744, 28.801703, 28.618474]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.820497,"[0.995171, 0.098156]"
28.76974,"[-0.098156, 0.995171]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
0.11709
-0.210939
0.011899
-0.162391
-0.050289


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.334676,1.002754,0.490268,0.484181,0.487205,0.503104,0.500475
"""a""",0.328401,0.985268,0.508709,0.512565,0.51063,0.504645,0.507109


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""hello"""
"""going"""
"""world"""
"""church"""
"""to"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""go"""
"""hello"""
"""world"""
"""church"""
""""""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.166248
1.339555
1.29705


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.819894,null
null,-0.106444,null
0.166248,0.311177,0.46741
1.339555,1.507627,1.895496
1.29705,-0.40433,0.528693


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""cC""",null
null,"""HxBp""",null
0.166248,"""qjl""","""RMFn"""
1.339555,"""3N""","""n6Al"""
1.29705,"""cHD""","""MF"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.166248,"""uhpES"""
1.339555,"""1AzJe"""
1.29705,"""EpZWF"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,0.175841,null,0.176231
null,-0.816172,null,-0.815861
0.166248,1.955628,1.995267,1.955387
1.339555,0.576981,3.011182,0.57688
1.29705,-1.083462,1.005481,-1.083934


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.714802,-0.54607,2
null,0.355513,0.827599,9
0.166248,0.002282,0.570296,0
1.339555,0.786958,-0.031061,3
1.29705,0.320615,-0.158668,8


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.425243,0.670722,0.782871,0.676086


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.598321,0.075415,4,5
1,0.073296,0.789893,2,3
2,0.818023,0.504974,2,4
0,0.985104,0.153053,0,6
1,0.440852,0.862906,2,1


In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.356596,0.721402}","{31.810658,0.668157}","{0.744296,0.5617}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.782406,0.434031}  ┆ {32.80012,0.621581}  ┆ {2.027486,0.088156} │
│ 1         ┆ {-1.168306,0.242767} ┆ {34.251982,0.551894} ┆ {0.414089,0.798598} │
│ 2         ┆ {0.988312,0.323072}  ┆ {35.722092,0.481702} ┆ {1.335438,0.254489} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
537,0.1074
534,0.1068
577,0.1154
605,0.121
546,0.1092
572,0.1144
505,0.101
561,0.1122
563,0.1126


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.720043,0.762057,0.0802,0.853373,1.184888,16
1,0.746859,0.774783,0.969885,0.027992,6.011372,15
2,0.21097,0.029106,0.522927,0.317476,9.596375,14
3,0.701792,0.527346,0.352297,0.912383,4.874474,16
4,0.723815,0.544753,0.311694,0.210474,5.696281,17


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.720043,0.762057,0.0802,0.853373,1.184888,690
1,0.746859,0.774783,0.969885,0.027992,6.011372,1
2,0.21097,0.029106,0.522927,0.317476,9.596375,56
3,0.701792,0.527346,0.352297,0.912383,4.874474,1289
4,0.723815,0.544753,0.311694,0.210474,5.696281,23


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.720043,0.762057,0.0802,0.853373,1.184888,"[0, 1171, … 1754]"
1,0.746859,0.774783,0.969885,0.027992,6.011372,"[1, 906, … 1751]"
2,0.21097,0.029106,0.522927,0.317476,9.596375,"[2, 50, … 853]"
3,0.701792,0.527346,0.352297,0.912383,4.874474,"[3, 1558, … 921]"
4,0.723815,0.544753,0.311694,0.210474,5.696281,"[4, 3, … 485]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 1171, … 912] ┆ 9                  │
│ 1   ┆ [1, 906, … 831]  ┆ 5                  │
│ 2   ┆ [2, 50, … 1682]  ┆ 8                  │
│ 3   ┆ [3, 1558, … 66]  ┆ 7                  │
│ 4   ┆ [4, 3, … 1370]   ┆ 6                  │
└─────┴──────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.720043,0.762057,0.0802,0.853373,1.184888,"[0, 1171, … 1754]","[0.0, 0.054683, … 0.077248]"
1,0.746859,0.774783,0.969885,0.027992,6.011372,"[1, 906, … 1751]","[0.0, 0.042337, … 0.053288]"
2,0.21097,0.029106,0.522927,0.317476,9.596375,"[2, 50, … 853]","[0.0, 0.059335, … 0.06505]"
3,0.701792,0.527346,0.352297,0.912383,4.874474,"[3, 1558, … 921]","[0.0, 0.015422, … 0.067852]"
4,0.723815,0.544753,0.311694,0.210474,5.696281,"[4, 3, … 485]","[0.0, 0.049363, … 0.060237]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
3,0.701792,0.527346,0.352297,0.912383,4.874474
4,0.723815,0.544753,0.311694,0.210474,5.696281
6,0.736724,0.776174,0.693574,0.532166,3.944928
7,0.617642,0.788939,0.488318,0.27519,2.900536
8,0.327207,0.456177,0.469014,0.128877,9.917232


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
36,0.568433,0.456969,0.05886,0.006766,0.753822
117,0.521121,0.521892,0.129322,0.377708,0.731702
138,0.531877,0.488015,0.040861,0.905918,1.466601
169,0.465704,0.501999,0.253489,0.625459,6.367421
176,0.473739,0.491308,0.894173,0.652556,6.084469


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
169,0.465704,0.501999,0.253489,0.625459,6.367421
176,0.473739,0.491308,0.894173,0.652556,6.084469
235,0.564589,0.484392,0.057102,0.975023,8.699902
367,0.478239,0.566379,0.620646,0.384922,9.836408
383,0.501891,0.46347,0.135889,0.616873,7.838947


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1345, … 304]",4
1,"[1, 6, 278]",3
2,"[2, 934, 853]",3
3,"[3, 1584, … 159]",5
4,"[4, 1939, … 392]",5


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [53]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_19683/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.78908,0,1,"""7J""","""k"""
1.0,0.503485,1,5,"""S""","""yj"""
1.0,0.736868,2,8,"""iB""","""p"""
1.0,0.904397,1,4,"""R""","""Js"""
1.0,0.843379,1,2,"""A""","""WR"""


In [54]:
df_pd = df.to_pandas()

In [55]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [56]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.503697  ┆ 0.500827 ┆ 0.502258 ┆ 0.50319           ┆ 0.501299 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.503697  ┆ 0.500827 ┆ 0.502258 ┆ 0.50319           ┆ 0.501299 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬───────

In [57]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.194464,0.2,0.187,0.000874
0.413897,0.2,0.207,0.000241
0.5999,0.2,0.19,0.000513
0.804026,0.2,0.202,0.00002
inf,0.2,0.214,0.000947


In [58]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
12.900387


In [59]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.123215


In [60]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
2
2
2
2
1
2
2
